In [4]:
#import libraries
import numpy as np
import imutils
import time
from scipy import spatial
import cv2
import os

In [5]:
#parameters
FRAMES_BEFORE_CURRENT = 10
preDefinedConfidence = 0.5
preDefinedThreshold = 0.5
inputWidth, inputHeight = 416, 416 #Experimental

In [6]:
#loading yolo
LABELS = open('./yolo/coco.names').read().strip().split("\n")
yolo_weights = './yolo/yolov3.weights'
yolo_cfg = './yolo/yolov3.cfg'

In [7]:
list_of_vehicles = ["person"]
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

In [8]:
#get frame and time
def countFPS(start_time, num_frames):
    current_time = int(time.time())
    if(current_time > start_time):
        os.system('clear')
        num_frames = 0
        start_time = current_time
    return start_time, num_frames

In [9]:
#draw rectangle on detected cars (chq if boxs exist->flatten the array->get corners for rec,draw rec and circle)
def drawDetectionBoxes(idxs, boxes, classIDs, confidences, frame):
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],confidences[i])
            cv2.putText(frame, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.circle(frame, (x + (w//2), y + (h//2)), 2, (0, 0xFF, 0), thickness=2)

In [10]:
#load yolo from disk
net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)

ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [11]:
previous_frame_detections = [{(0, 0): 0} for i in range(FRAMES_BEFORE_CURRENT)]
num_frames, vehicle_count = 0, 0
start_time = int(time.time())

In [12]:
video_dir = '../Shoplifting/Shoplifting001_x264.mp4'
for entry in os.listdir(video_dir):
    if os.path.isfile(os.path.join(video_dir, entry)):
        print(entry)
        cap = cv2.VideoCapture(video_dir+'/'+entry)
        video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        if cap.isOpened() == False:
            print('ERROR FILE NOT FOUND, OR WRONG CODEC USE!')
            
        while cap.isOpened():
            ret,frame = cap.read();
            if ret == True:
                num_frames += 1
                boxes, confidences, classIDs = [], [], []
                start_time, num_frames = countFPS(start_time, num_frames)
                blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (inputWidth, inputHeight),swapRB=True, crop=False)
                net.setInput(blob)
                start = time.time()
                layerOutputs = net.forward(ln)
                end = time.time()
                for output in layerOutputs:
                    for i, detection in enumerate(output):
                        scores = detection[5:]
                        classID = np.argmax(scores)
                        confidence = scores[classID]
                        if confidence > preDefinedConfidence:
                            box = detection[0:4] * np.array([video_width, video_height, video_width, video_height])
                            (centerX, centerY, width, height) = box.astype("int")
                            x = int(centerX - (width / 2))
                            y = int(centerY - (height / 2))
                            boxes.append([x, y, int(width), int(height)])
                            confidences.append(float(confidence))
                            classIDs.append(classID)

                idxs = cv2.dnn.NMSBoxes(boxes, confidences, preDefinedConfidence,preDefinedThreshold)
                drawDetectionBoxes(idxs, boxes, classIDs, confidences, frame)
                cv2.imshow(entry, frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                if (cv2.waitKey(10) & 0xFF == ord('q')):
                    break
            else:
                break


        print("[INFO] cleaning up...")
        cap.release()
        cv2.destroyAllWindows()

Shoplifting001_x264_0.mp4
[INFO] cleaning up...
Shoplifting001_x264_1.mp4
[INFO] cleaning up...
Shoplifting001_x264_10.mp4
[INFO] cleaning up...
Shoplifting001_x264_11.mp4
[INFO] cleaning up...
Shoplifting001_x264_12.mp4
[INFO] cleaning up...
Shoplifting001_x264_13.mp4
[INFO] cleaning up...
Shoplifting001_x264_14.mp4
[INFO] cleaning up...
Shoplifting001_x264_15.mp4
[INFO] cleaning up...
Shoplifting001_x264_16.mp4
[INFO] cleaning up...
Shoplifting001_x264_17.mp4
[INFO] cleaning up...
Shoplifting001_x264_18.mp4
[INFO] cleaning up...
Shoplifting001_x264_19.mp4
[INFO] cleaning up...
Shoplifting001_x264_2.mp4
[INFO] cleaning up...
Shoplifting001_x264_20.mp4
[INFO] cleaning up...
Shoplifting001_x264_21.mp4
[INFO] cleaning up...
Shoplifting001_x264_22.mp4
[INFO] cleaning up...
Shoplifting001_x264_23.mp4
[INFO] cleaning up...
Shoplifting001_x264_24.mp4
[INFO] cleaning up...
Shoplifting001_x264_25.mp4
[INFO] cleaning up...
Shoplifting001_x264_26.mp4
[INFO] cleaning up...
Shoplifting001_x264_27.